<a href="https://colab.research.google.com/github/kok554/computervision/blob/main/CNN_%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# CNN 모델 정의
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # Convolutional Layer 1: 입력 채널 1개, 출력 채널 10개, 필터 크기 5x5
    self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
    self.pool = nn.MaxPool2d(2, 2) # Max Pooling Layer: 2x2
    self.conv2 = nn.Conv2d(10, 20, kernel_size =5)  # Convolutional Layer 2: 출력 채널 20개
    self.fc1 = nn.Linear(20 * 4 * 4, 50) # Fully Connected Layer: 입력 -> 50차원
    self.fc2 = nn.Linear(50, 10)
    self.relu = nn.ReLU() # ReLU 활성화 함수
    self.softmax = nn.Softmax(dim=1) # Softmax 활성화 함수

  def forward(self, x):
    x = self.relu(self.pool(self.conv1(x))) # Conv1 -> ReLU -> Pooling
    x = self.relu(self.pool(self.conv2(x))) # Conv2 -> ReLU -> Pooling
    x = x.view(-1, 20 * 4 * 4) # 텐서를 펼침
    x = self.relu(self.fc1(x)) # FC1 -> ReLU
    x = self.fc2(x)  # FC2
    return self.softmax(x) # Softmax 출력

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(20 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.pool(self.conv1(x)))
        x = self.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 20 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

# 데이터셋 준비
# MNIST 데이터를 불러오고, 텐서로 변환 및 정규화
transform = transforms.Compose([
    transforms.ToTensor(),                          # 데이터를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))           # 데이터를 평균 0, 표준편차 1로 정규화
])

# 학습용 및 테스트용 MNIST 데이터셋 다운로드 및 로드
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)  # 학습 데이터
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)  # 테스트 데이터

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  # 학습 데이터 로더
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)  # 테스트 데이터 로더

# 모델, 손실 함수, 옵티마이저 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 가능 여부 확인
model = CNN().to(device)                                              # CNN 모델 생성 및 GPU로 이동
criterion = nn.CrossEntropyLoss()                                     # 다중 클래스 분류를 위한 손실 함수
optimizer = optim.Adam(model.parameters(), lr=0.001)                 # Adam 옵티마이저 설정

# 학습 함수
def train(model, device, train_loader, optimizer, criterion):
    model.train()  # 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):  # 배치 단위로 데이터 가져오기
        data, target = data.to(device), target.to(device)      # 데이터와 라벨을 GPU로 이동
        optimizer.zero_grad()                                  # 이전의 기울기 초기화
        output = model(data)                                   # 모델에 입력 데이터 전달하여 출력 계산
        loss = criterion(output, target)                      # 손실 계산
        loss.backward()                                        # 역전파로 기울기 계산
        optimizer.step()                                       # 옵티마이저로 가중치 갱신
        # 100번째 배치마다 학습 상태 출력
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 테스트 함수
def test(model, device, test_loader):
    model.eval()  # 모델을 평가 모드로 설정 (드롭아웃 등 비활성화)
    correct = 0  # 정확히 예측한 데이터 수를 저장하는 변수
    with torch.no_grad():  # 기울기 계산 비활성화 (속도 향상)
        for data, target in test_loader:  # 테스트 데이터 배치 단위로 가져오기
            data, target = data.to(device), target.to(device)  # 데이터와 라벨을 GPU로 이동
            output = model(data)                              # 모델에 입력 데이터 전달하여 출력 계산
            pred = output.argmax(dim=1, keepdim=True)         # 가장 높은 점수를 가진 클래스 예측
            correct += pred.eq(target.view_as(pred)).sum().item()  # 정답 개수 누적
    # 테스트 결과 출력
    print(f'\nTest set: Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

# 학습 및 테스트 반복
n_epochs = 5  # 에폭 수
for epoch in range(n_epochs):
    train(model, device, train_loader, optimizer, criterion)  # 학습 함수 호출
    test(model, device, test_loader)                         # 테스트 함수 호출

# 모델 저장 디렉토리 생성
os.makedirs('models', exist_ok=True)

# 학습된 모델 저장
torch.save(model.state_dict(), 'models/mnist_cnn_model.pth')  # 모델 가중치 저장
print("Model saved as 'models/mnist_cnn_model.pth'")          # 저장 완료 메시지 출력


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301521
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.665880
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.595212
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.634517
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.560084
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.659370
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.482914
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.568281
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.588163
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.539102

Test set: Accuracy: 8803/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.625563
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.591961
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.608555
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.606259
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.615196
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.497026
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.588341
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.585342
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.606048
Train Epoch: 2 [57600/6

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 데이터셋 변환 함수: MNIST -> 짝수/홀수 레이블
# 레이블(0~9)을 짝수(0), 홀수(1)로 변환
class EvenOddDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.dataset.targets = self.dataset.targets % 2

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]

# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(20 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.relu(self.pool(self.conv1(x)))
        x = self.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 20 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 전이학습용 수정된 CNN 모델 정의
class TransferCNN(nn.Module):
    def __init__(self, original_model):
        super(TransferCNN, self).__init__()
        self.conv1 = original_model.conv1
        self.conv2 = original_model.conv2
        self.pool = original_model.pool
        self.relu = original_model.relu
        self.fc1 = original_model.fc1
        self.fc2 = nn.Linear(50, 2)  # 출력 차원을 2로 변경 (짝수/홀수 분류)

    def forward(self, x):
        x = self.relu(self.pool(self.conv1(x)))
        x = self.relu(self.pool(self.conv2(x)))
        x = x.view(-1, 20 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 데이터 변환 및 로더 준비
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 원본 MNIST 데이터셋
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# 짝수/홀수 데이터셋 변환
train_dataset = EvenOddDataset(train_dataset)
test_dataset = EvenOddDataset(test_dataset)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 기존 학습된 CNN 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
original_model = CNN().to(device)
original_model.load_state_dict(torch.load('models/mnist_cnn_model.pth'))

# 전이학습용 수정된 모델 생성
transfer_model = TransferCNN(original_model).to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transfer_model.parameters(), lr=0.001)

# 학습 함수
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch {epoch + 1}, Batch {batch_idx}, Loss: {loss.item():.6f}")

# 테스트 함수
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"\nTest Accuracy: {accuracy:.2f}%\n")

# Case 1: Convolutional Layers Freeze
for param in transfer_model.conv1.parameters():
    param.requires_grad = False
for param in transfer_model.conv2.parameters():
    param.requires_grad = False

print("Training with Convolutional Layers Frozen...")
for epoch in range(5):
    train(transfer_model, device, train_loader, optimizer, criterion, epoch)
    test(transfer_model, device, test_loader)

# Case 2: Fully Connected Layer (fc1) Freeze 추가
for param in transfer_model.fc1.parameters():
    param.requires_grad = False

# 옵티마이저 재정의
optimizer = optim.Adam(filter(lambda p: p.requires_grad, transfer_model.parameters()), lr=0.001)

print("Training with Fully Connected Layer Frozen...")
for epoch in range(5):
    train(transfer_model, device, train_loader, optimizer, criterion, epoch)
    test(transfer_model, device, test_loader)

<ipython-input-7-396368fb2f30>:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  original_model.load_state_dict(torch.load('models/mnist_cnn_model.pth'))


Training with Convolutional Layers Frozen...
Epoch 1, Batch 0, Loss: 5.256649
Epoch 1, Batch 100, Loss: 0.013401
Epoch 1, Batch 200, Loss: 0.056751
Epoch 1, Batch 300, Loss: 0.024768
Epoch 1, Batch 400, Loss: 0.017111
Epoch 1, Batch 500, Loss: 0.042289
Epoch 1, Batch 600, Loss: 0.007125
Epoch 1, Batch 700, Loss: 0.008383
Epoch 1, Batch 800, Loss: 0.004868
Epoch 1, Batch 900, Loss: 0.017127

Test Accuracy: 99.29%

Epoch 2, Batch 0, Loss: 0.016011
Epoch 2, Batch 100, Loss: 0.128650
Epoch 2, Batch 200, Loss: 0.015584
Epoch 2, Batch 300, Loss: 0.004562
Epoch 2, Batch 400, Loss: 0.000661
Epoch 2, Batch 500, Loss: 0.009845
Epoch 2, Batch 600, Loss: 0.003260
Epoch 2, Batch 700, Loss: 0.014521
Epoch 2, Batch 800, Loss: 0.021021
Epoch 2, Batch 900, Loss: 0.005634

Test Accuracy: 99.46%

Epoch 3, Batch 0, Loss: 0.003057
Epoch 3, Batch 100, Loss: 0.001484
Epoch 3, Batch 200, Loss: 0.004154
Epoch 3, Batch 300, Loss: 0.003492
Epoch 3, Batch 400, Loss: 0.002544
Epoch 3, Batch 500, Loss: 0.060356
Epo